In [1]:
# NOTEBOOK IS MEANT TO BE RUN ENTIRE WAY THROUGH AFTER EACH ITERATION
# AUTOLOCKING OF DATA HELPS TO PRESERVE MODEL OUTPUT THAT MAY HAVE BEEN OVERWRITTEN ON EACH MODEL RUN
# INDIVIDUAL PURPOSES CAN BE LOCKED AT ANY ROUND AND THEY WILL DROP OUT OF ADJUSTMENT AND SMOOTHING

# debug toggle to turn on display of dataframes in cell outputs
debug = False

import pandas as pd  
import os
import plotly as py
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
import numpy as np
from scipy import special
import time

# offline mode for charts, so it stays local
py.offline.init_notebook_mode(connected=True)

# set rounding precision and display precision - display is rounding + 1, so you can see that it's working
round_precision = 7
pd.set_option("display.precision", round_precision + 1)

# read in curve data'
dirCWD    = os.getcwd()
dirIntermediate = os.path.join(dirCWD,'_Friction_Factors')
dirResults = os.path.join(dirCWD,'_Friction_Factors')

binsize = 2

observedStage = 'v9 Final Smoothing'

filenameTLFObs = r'A:\1 - TDM\2 - Estimate Param\_General Parameters\7 - Update Obs TLF\3 - Create Obs TLFs for all Trips\results\dfTLF_Obs_wSmoothed_20221130-133903.csv'
folderTDMTLFLoc = r'A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9.0Beta\WF TDM v9.0 - 2022-11-01\Scenarios\BY_2019\3_Distribute\TLF'

# some trip purposes are added together for information purposes
dfTripPurpSubtotals = pd.DataFrame([
    ['HBOth','HBO' ],
    ['HBShp','HBO' ],
    ['NHBW' ,'NHB' ],
    ['NHBNW','NHB' ],
    ['IX'   ,'IXXI'],
    ['XI'   ,'IXXI']
],columns=(['TRIPPURP','TRIPPURP_SUB']))
if debug: display(dfTripPurpSubtotals)

# calibration round settings
dfCalibrationRounds = pd.DataFrame([
    ['0-Initial from v832'    ,'_r0',r'A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9.0Beta\WF TDM v9.0 - 2022-11-01\1_Inputs\0_GlobalData\3_Distribute\FricFactor_AllPurp.csv'],
    ['1-First'                ,'_r1',r'A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9.0Beta\WF TDM v9.0 - 2022-11-01\1_Inputs\0_GlobalData\3_Distribute\FricFactor_AllPurp.csv'],
    ['2-no IXXI MH-HV changes','_r2',r'A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9.0Beta\WF TDM v9.0 - 2022-11-01\1_Inputs\0_GlobalData\3_Distribute\FricFactor_AllPurp.csv'],
    ['3-Third Time'           ,'_r3',r'A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9.0Beta\WF TDM v9.0 - 2022-11-01\1_Inputs\0_GlobalData\3_Distribute\FricFactor_AllPurp.csv']
],columns=(['FF_CALIB_ROUND','TDM_TLF_FOLDER','FILENAMEFF']))

# create and initialize locking fields
dfCalibrationRounds['LOCKFF' ] = False
dfCalibrationRounds['LOCKTLF'] = False

# AUTOLOCK ALL BUT LAST CALIBROUND TO AVOID OVERWRITING DATA IF NOT MANUALLY LOCKED
dfCalibrationRounds.loc[(dfCalibrationRounds.index<dfCalibrationRounds.shape[0]-1), 'LOCKFF' ] = True
dfCalibrationRounds.loc[(dfCalibrationRounds.index<dfCalibrationRounds.shape[0]-1), 'LOCKTLF'] = True

display(dfCalibrationRounds)

# add trip purposes here to lock friction factors
dfTripPurpLocks = pd.DataFrame([
    #example: ['HBW','0-Initial from v832'],
    ['NHBW' ,1],
    ['NHBNW',1],
    ['HBOth',1]
#    ['IX_MD','0-Initial from v832'],
#    ['IX_HV','0-Initial from v832'],
#    ['XI'   ,'0-Initial from v832'],
#    ['XI_MD','0-Initial from v832'],
#    ['XI_HV','0-Initial from v832'],
],
columns=('TRIPPURP','LOCK_FF_CALIB_ROUND_INDEX'))
if debug: display(dfTripPurpLocks)

# a set of column renaming to be used to put all columns in consistent naming
colRenames ={'HBOTH':'HBOth',
             'HBSHP':'HBShp',
             'HBSCHPR' :'HBSchPr',
             'HBSCHSC' :'HBSchSc',
             'HBSCH_PR':'HBSchPr',
             'HBSCH_SC':'HBSchSc',
             'HBSch_Pr':'HBSchPr',
             'HBSch_Sc':'HBSchSc'}

# max bin for normalizing adjusted FF (setting max=1)... in bins higher than this value, any value greater than 1.0 will be set to 1.0
nrmlz_max_bin = 100

# use entire width of browser for cells
#from IPython.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

,FF_CALIB_ROUND,TDM_TLF_FOLDER,FILENAMEFF,LOCKFF,LOCKTLF
0,0-Initial from v832,_r0,A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9...,True,True
1,1-First,_r1,A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9...,True,True
2,2-no IXXI MH-HV changes,_r2,A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9...,True,True
3,3-Third Time,_r3,A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9...,False,False


In [2]:
# create intermediate directories for each round
for index, row in dfCalibrationRounds.iterrows():
    path_ffcb = os.path.join(dirIntermediate,str(index))

    # Check whether the specified path exists or not
    isExist = os.path.exists(path_ffcb)
    if not isExist:
        os.makedirs(path_ffcb)
        print("The new directory is created! " + path_ffcb)

    path_ms = os.path.join(path_ffcb,'manual_smoothing')

    # Check whether the specified path exists or not
    isExist = os.path.exists(path_ms)
    if not isExist:
        os.makedirs(path_ms)
        print("The new directory is created! " + path_ms)

    path_fnr = os.path.join(path_ffcb,'factors for next round')
    isExist = os.path.exists(path_fnr)
    
    #printing if the path exists or not
    if not isExist:
        os.makedirs(path_fnr)
        print("The new directory is created! " + path_fnr)


# Observed Trip Length Frequencies

In [3]:
# read in observed TLF data to be used
dfTLF_Obs = pd.read_csv(filenameTLFObs)

#filter by the STAGE for 
dfTLF_Obs = dfTLF_Obs[dfTLF_Obs['STAGE']==observedStage].copy()
dfTLF_Obs = dfTLF_Obs.drop(columns=('STAGE'))
if debug: display(dfTLF_Obs)

In [4]:
# show max bin size for TLF type to use in updating TDM TLF code
if debug: display(dfTLF_Obs[dfTLF_Obs['FREQ']>0].groupby(['TLFTYPE'],as_index=False).agg(MAXBIN=('BIN','max')))

In [5]:
# Check TLF Observed. FREQ should all sum to 1! Also show average trip length
dfTLF_Obs['BINxFREQ'] = dfTLF_Obs['BIN'] * dfTLF_Obs['FREQ']
dfTLF_Obs_Stats = dfTLF_Obs.groupby(['TRIPPURP','TLFTYPE'],as_index=False).agg(FREQ_SUM=('FREQ','sum'),AVG_TRIP_LEN=('BINxFREQ','sum'))
dfTLF_Obs.drop(columns=('BINxFREQ'), inplace=True)
if debug: display(dfTLF_Obs_Stats)

# Observed Plots

In [6]:
#PLOTTING FUNCTION

# chart preset zoom extents
dfZoomPresets = pd.DataFrame([
    ['Begin' ,     0,   100,     0,np.NaN],
    ['Mid'   ,    30,   160,     0, 0.018],
    ['Tail'  ,   100,   400,     0, 0.001],
    ['All'   ,np.NaN,np.NaN,np.NaN,np.NaN],
    ['Custom',np.NaN,np.NaN,np.NaN,np.NaN],
], columns=('PRESET','XMIN','XMAX','YMIN','YMAX'))
dfZoomPresets

def update_plot_obs(trippurps, tlftypes, zoompreset, xmin, xmax, ymin, ymax):

    dfAvgTripLen = pd.DataFrame()
    
    data = []
    for trippurp in trippurps:
        for tlftype in tlftypes:

            # data for plotting from filtered dataframe
            plotdata = dfTLF_Obs[(dfTLF_Obs['TRIPPURP']==trippurp) & (dfTLF_Obs['TLFTYPE']==tlftype)].copy()

            # fill any NaN values with zeros
            plotdata = plotdata.fillna(0)
            plotdata = plotdata.rename(columns={'FREQ_Obs':'FREQ'}) # temp fix, since this column gets renamed somewhere else
            
            xplot = plotdata['BIN']
            yplot = plotdata['FREQ']
            name  = trippurp + ' ' + tlftype 

            trace1 = go.Scatter(
                x=xplot,
                y=yplot,
                mode='lines',
                name=name,
                line=dict(
                    shape='spline'
                )
            )
            data.append(trace1)
                
    # get zoom presets
    if (zoompreset=='Custom'):
        _xmin = xmin
        _xmax = xmax
        _ymin = ymin
        _ymax = ymax
    else:
        _xmin = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['XMIN'].values[0]
        _xmax = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['XMAX'].values[0]
        _ymin = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['YMIN'].values[0]
        _ymax = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['YMAX'].values[0]

    layout = go.Layout(
        title='Trip Length Frequencies - Observed Data',
        yaxis=dict(
            title='',
            range=(_ymin,_ymax)
        ),
        xaxis=dict(
            title='GC/Dist/Time',
            range=(_xmin,_xmax)
        ),
        #width=1600,
        height=450
    )
    
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)

In [7]:
# MAKE INTERACTIVE CHART
py.offline.init_notebook_mode(connected=True)

lstTripPurp  = dfTLF_Obs['TRIPPURP'].unique().tolist() # HBW, HBShp, HBOth,...
lstTLFType   = dfTLF_Obs['TLFTYPE' ].unique().tolist() # GC, Time, Dist
    
selectTripPurp  = widgets.SelectMultiple(options=lstTripPurp, value=('HBW'        ,), description='Trip Purpose')
selectTLFType   = widgets.SelectMultiple(options=lstTLFType , value=('GC'         ,), description='TLF Type'    )
selectPreset    = widgets.Select(options=dfZoomPresets['PRESET'].tolist(), value='All', description='Zoom Presets')

#custom extents
xmin = widgets.Text(value='0', description='X Min')
xmax = widgets.Text(value='60', description='X Max')
ymin = widgets.Text(value='0', description='Y Min')
ymax = widgets.Text(value='.01', description='Y Max')

widgets.interactive(update_plot_obs, trippurps=selectTripPurp, tlftypes=selectTLFType, zoompreset=selectPreset, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)

interactive(children=(SelectMultiple(description='Trip Purpose', index=(0,), options=('HBW', 'HBShp', 'HBOth',…

# Friction Factors

In [8]:
# read Friction Factors into array ordered by iteration

# Friction Factors array that stores a separate dataframe for each Calibration Round
dfsFF=[]

for index, row in dfCalibrationRounds.iterrows():
    filename = os.path.join(os.path.join(dirIntermediate,str(index)),'1-friction factors begin.csv')
    if row['LOCKFF']==False:
        display ('Reading friction factors for ' + row['FF_CALIB_ROUND'])
        dfFF = pd.read_csv(row['FILENAMEFF'])

        # make sure id column is always same: BIN
        dfFF.rename(columns={';BIN':'BIN',';MINUTE':'BIN','Mile':'BIN','Mil':'BIN','Min':'BIN','Bin':'BIN'},inplace=True)
        # rename columns
        dfFF.rename(columns=colRenames,inplace=True)

        if debug: display(dfFF)

        # melt all columns but first
        dfFF = pd.melt(dfFF, id_vars=['BIN'], value_vars=dfFF.columns[1:].tolist(), ignore_index=False, var_name='TRIPPURP', value_name='FF')

        dfFF.to_csv(filename, index=False)

    else:
        display ('Reading csv for ' + row['FF_CALIB_ROUND'])
        dfFF = pd.read_csv(filename)

    # append to array of Friction Factors
    dfsFF.append(dfFF)

    if debug: display(dfFF)


'Reading csv for 0-Initial from v832'

'Reading csv for 1-First'

'Reading csv for 2-no IXXI MH-HV changes'

'Reading friction factors for 3-Third Time'

# Read in TDM Trip Length Frequencies

In [9]:
dfCalibrationRounds

,FF_CALIB_ROUND,TDM_TLF_FOLDER,FILENAMEFF,LOCKFF,LOCKTLF
0,0-Initial from v832,_r0,A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9...,True,True
1,1-First,_r1,A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9...,True,True
2,2-no IXXI MH-HV changes,_r2,A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9...,True,True
3,3-Third Time,_r3,A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9...,False,False


In [9]:
# read in trips from TDM sorted into TLF bins

# array of dataframes to hold TLFs from each round of calibration
dfsTLF_TDM = []

for index, row in dfCalibrationRounds.iterrows():
    
    filename = os.path.join(os.path.join(dirIntermediate,str(index)),'2-tlfs.csv')

    if row['LOCKTLF']==False:

        display('Reading ' + row['FF_CALIB_ROUND'] + ' TLFs')

        tlfs=['Cost','Dist','Time']

        dfTLFTDMTrips = pd.DataFrame()

        for tlf in tlfs:
            
            # read in csv for tlf
            dfRead = pd.read_csv(os.path.join(folderTDMTLFLoc, row['TDM_TLF_FOLDER'] + '\\TLF_' + tlf + '.csv'))
            
            # make sure id column is always same: BIN
            dfRead.rename(columns={';BIN':'BIN',';MINUTE':'BIN','Mile':'BIN','Mil':'BIN','Min':'BIN','Bin':'BIN'}, inplace=True)
            # rename columns
            dfRead.rename(columns=colRenames,inplace=True)

            dfRead = pd.melt(dfRead, id_vars=['BIN'], value_vars=dfRead.columns[1:].tolist(), ignore_index=False, var_name='TRIPPURP', value_name='TRIPS')

            # set TLF value, rename Cost to GC
            if tlf=='Cost':
                dfRead['TLFTYPE'] = 'GC'
            else:
                dfRead['TLFTYPE'] = tlf

            # reorder columns
            dfRead = dfRead[['TLFTYPE','TRIPPURP','BIN','TRIPS']]

            # concat data into single dataframe
            dfTLFTDMTrips = pd.concat([dfTLFTDMTrips, dfRead], ignore_index=True)

        if debug: display(dfTLFTDMTrips)

        # create subtotals
        dfTLFTDMTripsForSubtotals = pd.DataFrame.merge(dfTLFTDMTrips, dfTripPurpSubtotals, on=('TRIPPURP'))
        dfTLFTDMTripsForSubtotals = dfTLFTDMTripsForSubtotals.groupby(['TLFTYPE','TRIPPURP_SUB','BIN'],as_index=False).agg(TRIPS=('TRIPS','sum'),COUNT=('TRIPS','size'))
        dfTLFTDMTripsForSubtotals = dfTLFTDMTripsForSubtotals[['TLFTYPE','TRIPPURP_SUB','BIN','TRIPS']]
        dfTLFTDMTripsForSubtotals = dfTLFTDMTripsForSubtotals.rename(columns={'TRIPPURP_SUB':'TRIPPURP'})
        if debug: display (dfTLFTDMTripsForSubtotals)

        # concat subtotals
        dfTLFTDMTrips = pd.concat([dfTLFTDMTrips,dfTLFTDMTripsForSubtotals], ignore_index=True)
        if debug: display(dfTLFTDMTrips)

        # calculated collapsed bins
        from math import floor
        def round_to_binsize(x):
            return int(binsize * floor(float(x)/binsize))
        dfTLFTDMTrips['BIN_COLLAPSE'] = dfTLFTDMTrips['BIN'].apply(lambda x: round_to_binsize(x))
        if debug: display(dfTLFTDMTrips)

        # aggregate to collapsed bins
        dfTLFTDMTrips = dfTLFTDMTrips.groupby(['TLFTYPE','TRIPPURP','BIN_COLLAPSE'], as_index=False).agg(TRIPS=('TRIPS','sum'))
        dfTLFTDMTrips = dfTLFTDMTrips.rename(columns={'BIN_COLLAPSE':'BIN'})
        if debug: display(dfTLFTDMTrips)

        # calculate percent distribution
        dfTLFTDMTripTotals = dfTLFTDMTrips.groupby(['TLFTYPE','TRIPPURP'], as_index=False).agg(TRIP_TOTAL=('TRIPS','sum'))
        if debug: display(dfTLFTDMTripTotals)

        # join to toals
        dfTLFTDMTripDist = pd.DataFrame.merge(dfTLFTDMTrips, dfTLFTDMTripTotals, on=('TLFTYPE','TRIPPURP'))
        dfTLFTDMTripDist['FREQ'] = dfTLFTDMTripDist['TRIPS'] / dfTLFTDMTripDist['TRIP_TOTAL']
        if debug: display(dfTLFTDMTripDist)

        dfTLF_TDM = dfTLFTDMTripDist[['TLFTYPE','TRIPPURP','BIN','FREQ']]
        if debug: display(dfTLF_TDM)

        # check to see if add up to 1.0
        dfCheck = dfTLF_TDM.groupby(['TLFTYPE','TRIPPURP'], as_index=False).agg(FREQ_SUM=('FREQ','sum'))
        # only display results out of range
        display('Not adding to 1 (if empty dataset, YAY!!!):')
        display(dfCheck[(dfCheck['FREQ_SUM']<0.9999999) | (dfCheck['FREQ_SUM']>1.0000001)])

        dfTLF_TDM.to_csv(filename, index=False)

    else:
        display('Reading TLFs csv for ' + row['FF_CALIB_ROUND'])
        dfTLF_TDM = pd.read_csv(filename)

    #add calibration round back into table
    dfTLF_TDM['FF_CALIB_ROUND'] = row['FF_CALIB_ROUND']

    # append dataframe to array
    dfsTLF_TDM.append(dfTLF_TDM)


'Reading TLFs csv for 0-Initial from v832'

'Reading TLFs csv for 1-First'

'Reading TLFs csv for 2-no IXXI MH-HV changes'

'Reading 3-Third Time TLFs'

'Not adding to 1 (if empty dataset, YAY!!!):'

,TLFTYPE,TRIPPURP,FREQ_SUM


C:\Users\cday\AppData\Local\ESRI\conda\envs\arcgispro-py3-plotly\lib\site-packages\ipykernel_launcher.py:92: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Model vs Observed Plots

Compare Average Trip Length Between Modeled and Observed Data

In [10]:
dfTLF_Obs_N = dfTLF_Obs
dfTLF_Obs_N['FF_CALIB_ROUND'] = 'Observed'
dfTLF_Obs_N.rename(columns={'FREQ_Obs':'FREQ'},inplace=True) # temp fix

#Merge together all Modeled data and Observed data
dfObsModelMerge = (pd.concat(dfsTLF_TDM).append(dfTLF_Obs_N))   

#CALCUATE TRIP LENGTH AVERAGES AS TABLE
dfObsModelMerge['BINxFREQ'] = dfObsModelMerge['BIN'] * dfObsModelMerge['FREQ']
dfObsModelMerge_Stats = (dfObsModelMerge
    .groupby(['FF_CALIB_ROUND','TRIPPURP','TLFTYPE'],as_index=False)
        .agg(FREQ_SUM=('FREQ','sum'),AVG_TRIP_LEN=('BINxFREQ','sum')))

if debug: display(dfObsModelMerge_Stats)

In [11]:
def update_df_mod_vs_obs(modeloption,tlfoption,trippurpoption):
    modellist = list(modeloption)
    
    colorlist = []
    for i in range(len(modellist)):
        if i == (len(modellist) - 1):
            colorlist.append('blue')
        else:
            colorlist.append('cornflowerblue')
    
    # append observed
    modellist.append('Observed')
    colorlist.append('red')

    dfObsModelMerge_Stats2 =(dfObsModelMerge_Stats
        .loc[dfObsModelMerge_Stats['FF_CALIB_ROUND'].isin(modellist)])
    dfObsModelMerge_Stats3 = (dfObsModelMerge_Stats2
        .loc[dfObsModelMerge_Stats2['TLFTYPE'].isin(tlfoption)])
    dfObsModelMerge_Stats4 = (dfObsModelMerge_Stats3
        .loc[dfObsModelMerge_Stats3['TRIPPURP'].isin(trippurpoption)])

    print(modellist[len(modellist)-2])

    if len(modellist) > 2:
        dfTLF_MvO_Wide2 = (dfObsModelMerge_Stats4
            .pivot(index=['TRIPPURP','TLFTYPE'],columns="FF_CALIB_ROUND",values="AVG_TRIP_LEN")
            .reset_index(['TRIPPURP','TLFTYPE'])
            .assign(PercentDiff = lambda x: (x[modellist[len(modellist)-2]] - x['Observed'])/(x['Observed'])))
    else:
        dfTLF_MvO_Wide2 = (dfObsModelMerge_Stats4
            .pivot(index=['TRIPPURP','TLFTYPE'],columns="FF_CALIB_ROUND",values="AVG_TRIP_LEN")
            .reset_index(['TRIPPURP','TLFTYPE'])
            .assign(PercentDiff = lambda x: (x[modellist[0]] - x['Observed'])/(x['Observed'])))

    fig = go.Figure(data=[go.Table(
                    header = dict(values=list(dfTLF_MvO_Wide2.columns),
                                  fill_color='paleturquoise',
                                  align='left'),
                    cells =  dict(values=round(dfTLF_MvO_Wide2,2).transpose().values.tolist(),
                                  fill_color=['lightgrey','lightgrey','lightgrey','lightgrey','lightgrey','lavender'],
                                  align='left'))
                    ])

    figbar2 = px.bar(dfObsModelMerge_Stats4, 
                     x = 'TRIPPURP', 
                     y = 'AVG_TRIP_LEN', 
                     color = 'FF_CALIB_ROUND', 
                     barmode = 'group',
                     color_discrete_sequence=colorlist
                    )

    figbar2.show()
    py.offline.iplot(fig)  

In [12]:
py.offline.init_notebook_mode(connected=True)

lstModOpt  = dfObsModelMerge['FF_CALIB_ROUND'].unique().tolist()
lstModOptShort = [round for round in lstModOpt if round != 'Observed']  
lstTLFType   = dfObsModelMerge['TLFTYPE'].unique().tolist() # GC, Time, Dist
lstTripPurp  = dfTLF_Obs['TRIPPURP'].unique().tolist() # HBW, HBShp, HBOth,...
    
selectModelOption  = widgets.SelectMultiple(options=lstModOptShort, value=lstModOptShort, description='Model Run'   )
selectTLFOption    = widgets.SelectMultiple(options=lstTLFType    , value=('GC' ,)      , description='TLF Type'    )
selectTripPurp     = widgets.SelectMultiple(options=lstTripPurp   , value=lstTripPurp   , description='Trip Purpose')

widgets.interactive(update_df_mod_vs_obs,modeloption=selectModelOption, tlfoption=selectTLFOption, trippurpoption=selectTripPurp)

#OBSERVED RED COLOR

interactive(children=(SelectMultiple(description='Model Run', index=(0, 1, 2, 3), options=('0-Initial from v83…

Comparing Trip Length Frequency Graphs between Modeled and Observed

In [13]:
#PLOTTING FUNCTION

# chart preset zoom extents
dfZoomPresets = pd.DataFrame([
    ['Begin' ,     0,   100,     0, np.NaN],
    ['Mid'   ,    30,   160,     0, 0.0018],
    ['Tail'  ,   100,   200,     0,0.0001],
    ['All'   ,np.NaN,200,np.NaN, np.NaN],
    ['Custom',np.NaN,np.NaN,np.NaN, np.NaN],
], columns=('PRESET','XMIN','XMAX','YMIN','YMAX'))
dfZoomPresets

def update_plot_mod_vs_obs(modeloption, trippurps, tlftypes, zoompreset, xmin, xmax, ymin, ymax):

    data = []
    for modelspace in modeloption:
        for trippurp in trippurps:
            for tlftype in tlftypes:
            
                # data for plotting from filtered dataframe
                plotdata1 = dfObsModelMerge[(dfObsModelMerge['TRIPPURP']==trippurp) & (dfObsModelMerge['TLFTYPE']==tlftype)]
                plotdata = plotdata1[(plotdata1['FF_CALIB_ROUND']==modelspace)]

                # fill any NaN values with zeros
                plotdata = plotdata.fillna(0)

                xplot = plotdata['BIN']
                yplot = plotdata['FREQ']
                name  = trippurp + ' ' + tlftype + ' (' + modelspace + ')'

                trace1 = go.Scatter(
                    x=xplot,
                    y=yplot,
                    mode='lines',
                    name=name,
                    line=dict(
                        shape='spline'
                    )
                )
                data.append(trace1)
                
    # get zoom presets
    if (zoompreset=='Custom'):
        _xmin = xmin
        _xmax = xmax
        _ymin = ymin
        _ymax = ymax
    else:
        _xmin = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['XMIN'].values[0]
        _xmax = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['XMAX'].values[0]
        _ymin = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['YMIN'].values[0]
        _ymax = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['YMAX'].values[0]

    layout = go.Layout(
        title='Trip Length Frequencies - Observed Data',
        yaxis=dict(
            title='',
            range=(_ymin,_ymax)
        ),
        xaxis=dict(
            title='GC/Dist/Time',
            range=(_xmin,_xmax)
        ),
        #width=1600,
        height=450
    )
    
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)

In [14]:
# MAKE INTERACTIVE CHART
py.offline.init_notebook_mode(connected=True)

lstTripPurp  = dfObsModelMerge['TRIPPURP'].unique().tolist() # HBW, HBShp, HBOth,...
lstTLFType   = dfObsModelMerge['TLFTYPE' ].unique().tolist() # GC, Time, Dist
lstModOpt    = dfObsModelMerge['FF_CALIB_ROUND'].unique().tolist()
    
selectTripPurp     = widgets.SelectMultiple(options=lstTripPurp                     , value=('HBW',)               , description='Trip Purpose')
selectTLFType      = widgets.SelectMultiple(options=lstTLFType                      , value=('GC' ,)               , description='TLF Type'    )
selectPreset       = widgets.Select        (options=dfZoomPresets['PRESET'].tolist(), value= 'Begin'                 , description='Zoom Presets')
selectModelOption  = widgets.SelectMultiple(options=lstModOpt                       , value=('0-Initial from v832',), description='Model Run')

#custom extents
xmin = widgets.Text(value='0'  , description='X Min')
xmax = widgets.Text(value='160' , description='X Max')
ymin = widgets.Text(value='0'  , description='Y Min')
ymax = widgets.Text(value='.10', description='Y Max')

widgets.interactive(update_plot_mod_vs_obs, modeloption=selectModelOption,trippurps=selectTripPurp, tlftypes=selectTLFType, zoompreset=selectPreset, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)

#ADD RAW OBSERVED CURVES

interactive(children=(SelectMultiple(description='Model Run', index=(0,), options=('0-Initial from v832', '1-F…

# Friction Factor Adjustment and Smoothing

In [15]:
# function check which trip urposes are unlocked for a given round
 
def getUnlockedListForCalibRound(roundindex):

    aUnlockedTripPurp = []

    # list trip purposes. must have dfsFF[0] loaded with complete set
    trippurps = dfsFF[0]['TRIPPURP'].unique().tolist()
    
    if debug: print('Calib Round Index: ' + str(roundindex))

    for trippurp in trippurps:
        
        #get round of lock
        lockedRoundIndices = dfTripPurpLocks[dfTripPurpLocks['TRIPPURP']==trippurp]['LOCK_FF_CALIB_ROUND_INDEX'].values

        if lockedRoundIndices.size > 0:
            lockedRoundIndex = lockedRoundIndices[0]
            if roundindex < lockedRoundIndex:
                aUnlockedTripPurp.append(trippurp)
        else:
            aUnlockedTripPurp.append(trippurp)

    return aUnlockedTripPurp

#display(getUnlockedListForCalibRound(2))

In [16]:
# calculated new friction factors for each round of calibration

# dataframe array of adjusted friction factors
dfsFFAdj=[]

for index, row in dfCalibrationRounds.iterrows():
    print('Calbration Round: ' + str(index))

    filename = os.path.join(os.path.join(dirIntermediate,str(index)),'3-friction factors adjusted.csv')
    
    # if friction factors locked
    if row['LOCKFF']==True:
        dfsFFAdj.append(pd.read_csv(filename))

    # if friction factors not locked
    else:

        # initialize dataframes with round data, round index should be same as indexes for TLF and FF
        dfTLF_TDM = dfsTLF_TDM[index][dfsTLF_TDM[index]['TLFTYPE']=='GC'].copy()
        dfFF      = dfsFF[index].copy()

        # filter out locked
        dfTLF_TDM = dfTLF_TDM[dfTLF_TDM['TRIPPURP'].isin(getUnlockedListForCalibRound(index))].copy()
        dfFF      = dfFF     [dfFF     ['TRIPPURP'].isin(getUnlockedListForCalibRound(index))].copy()
        
        # calculate adjustment factor
        dfTLF_TDM.rename(columns={'FREQ':'FREQ_TDM'},inplace=True)
        dfTLF_Obs.rename(columns={'FREQ':'FREQ_Obs'},inplace=True)

        dfModVsObs = pd.DataFrame.merge(dfTLF_TDM,dfTLF_Obs,on=('TRIPPURP','TLFTYPE','BIN'))

        # adjustment factor is observed divided by model
        dfModVsObs.loc[dfModVsObs['FREQ_TDM']>0 , 'ADJFACTOR'] = dfModVsObs['FREQ_Obs'] / dfModVsObs['FREQ_TDM']

        # if BIN 0 is empty, then use BIN 2 factor, unless empty as well
        dfModVsObs['ADJFACTOR_NEXTBIN'] = dfModVsObs['ADJFACTOR'].shift(-1) # create temp field for next bin factor... should be ok to not filter by TRIPPURP and TLFTYPE since ends of curves are always 0
        if debug: display(dfModVsObs)
        
        dfModVsObs.fillna(0,inplace=True) # fill NaNs with zeros
        
        dfModVsObs['ADJFACTOR'] = dfModVsObs.apply(lambda x: x['ADJFACTOR_NEXTBIN'] if (x['BIN']==0) & (x['ADJFACTOR']==0) & (x['ADJFACTOR_NEXTBIN']>=0) else x['ADJFACTOR'] ,axis=1)
        
        dfModVsObs.drop(columns=('ADJFACTOR_NEXTBIN'),inplace=True) # drop next bin factor column
        
        if debug: display(dfModVsObs)      

        # don't know if this is needed 
        #dfModVsObs.loc[dfModVsObs['FREQ_TDM']==0, 'ADJFACTOR'] = 1

        # raw adjusted friction factor: multiply friction factor from previous round by adjustment factor
        dfModVsObsWithFF = pd.DataFrame.merge(dfModVsObs,dfFF,on=('TRIPPURP','BIN'))

        dfModVsObsWithFF['FF_ADJ_RAW'] = dfModVsObsWithFF['FF'] * dfModVsObsWithFF['ADJFACTOR']

        # normalized adj friction factor: divide by max (round to seven decimals) of first 100 bins (because trucks have some real tails, so just want beginning)
        dfModVsObsWithFF_MaxAdj = (dfModVsObsWithFF[dfModVsObsWithFF['BIN']<=nrmlz_max_bin]
            .groupby(['TLFTYPE','TRIPPURP'], as_index=False)
                .agg(FF_ADJ_MAX=('FF_ADJ_RAW','max')))

        dfModVsObsWithFFAdjNrml = pd.DataFrame.merge(dfModVsObsWithFF,dfModVsObsWithFF_MaxAdj, on=('TLFTYPE','TRIPPURP'))

        dfModVsObsWithFFAdjNrml['FF_ADJ'] = round(dfModVsObsWithFFAdjNrml['FF_ADJ_RAW'] / dfModVsObsWithFFAdjNrml['FF_ADJ_MAX'],round_precision)

        #cap adjustment at 1.0 (for those beyond bin nrmlz_max_bin)
        dfModVsObsWithFFAdjNrml.loc[dfModVsObsWithFFAdjNrml['FF_ADJ']>1, 'FF_ADJ'] = 1

        dfModVsObsWithFFAdjNrml = dfModVsObsWithFFAdjNrml.rename(columns=({'FF':'FF_ORIG'}))

        # write out intermediate csv        
        dfFFAdj = dfModVsObsWithFFAdjNrml[['TRIPPURP','TLFTYPE','BIN','FF_ORIG','FF_ADJ']]
        dfFFAdj.to_csv(filename, index=False)
                
        if debug:
            display(dfModVsObsWithFFAdjNrml)
            dfCheck = (dfModVsObsWithFFAdjNrml
                .groupby(['TLFTYPE','TRIPPURP'])
                    .agg(FF_ADJ_MAX=('FF_ADJ','max')))
            print('These all should be 1:')
            display(dfCheck)

        # append to dataframe
        dfsFFAdj.append(dfFFAdj)
        
    if debug: display(dfsFFAdj)

Calbration Round: 0
Calbration Round: 1
Calbration Round: 2
Calbration Round: 3


In [34]:
#PLOTTING FUNCTION
# chart preset zoom extents
dfZoomPresets = pd.DataFrame([
    ['Begin' ,     0,    30,     0,np.NaN],
    ['Mid'   ,     0,   200,     0, 0.03],
    ['Tail'  ,   150,   400,     0, 0.001],
    ['All'   ,np.NaN,np.NaN,np.NaN,np.NaN],
    ['Custom',np.NaN,np.NaN,np.NaN,np.NaN],
], columns=('PRESET','XMIN','XMAX','YMIN','YMAX'))
dfZoomPresets

aOAS=['Original','Adjusted','Smoothed']
linewidths=[1.5,4,2]
linedashes=['solid','dash','solid']

#def update_plot_ff(calibrnds, trippurps, oass, zoompreset):
def update_plot_ff(calibrnds, trippurps, oass, zoompreset, xmin, xmax, ymin, ymax):

    dfAvgTripLen = pd.DataFrame()
    
    data = []
    for calibrnd in calibrnds:
        for oas in oass:
            
            linewidth = linewidths[aOAS.index(oas)]
            linedash  = linedashes[aOAS.index(oas)]

            for trippurp in trippurps:

                # data for plotting from filtered dataframe
                plotdata = dfFFsForCharts[(dfFFsForCharts['FF_CALIB_ROUND']==calibrnd) & (dfFFsForCharts['TRIPPURP']==trippurp) & (dfFFsForCharts['OAS']==oas) & (dfFFsForCharts['BIN']<400)]

                # fill any NaN values with zeros
                plotdata = plotdata.fillna(0)
                
                xplot = plotdata['BIN']
                yplot = plotdata['FF']
                name  = str(calibrnd) + ' ' + trippurp + ' ' + oas

                trace1 = go.Scatter(
                    x=xplot,
                    y=yplot,
                    mode='lines',
                    name=name,
                    line=dict(
                        shape='spline',
                        width=linewidth,
                        dash=linedash
                    )
                )
                data.append(trace1)
                
    # get zoom presets
    if (zoompreset=='Custom'):
        _xmin = xmin
        _xmax = xmax
        _ymin = ymin
        _ymax = ymax
    else:
        _xmin = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['XMIN'].values[0]
        _xmax = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['XMAX'].values[0]
        _ymin = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['YMIN'].values[0]
        _ymax = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['YMAX'].values[0]

    layout = go.Layout(
        title='Friction Factors',
        yaxis=dict(
            title='',
            range=(_ymin,_ymax)
        ),
        xaxis=dict(
            title='GC',
            range=(_xmin,_xmax)
        ),
        #width=1600,
        height=600
    )
    
    fig = go.Figure(data=data, layout=layout)
    #py.offline.iplot(fig)
    fig.show()

In [110]:
# SMOOTHING

# backup will output a new CSV each time function is run... just to be safe
backup = False

dfSmoothedSetHolder = pd.DataFrame()
dfFFsForCharts = pd.DataFrame() # master dataframe

# loop through input dataframe
for index, row in dfCalibrationRounds.iterrows():
    
    rndindex = index

    # get trip purposes and tlf types for looping
    tps  = dfsFF[rndindex]['TRIPPURP'].unique().tolist()

    dfSmoothParams = pd.DataFrame()

    # r=root, d=directories, f = files
    for r, d, f in os.walk(os.path.join(os.path.join(os.path.join(dirIntermediate,str(index)),'manual_smoothing'))):
        for file in f:
            if file.endswith('.csv'):
                dfSmoothParams = pd.concat([dfSmoothParams,pd.read_csv(os.path.join(r, file))])
    if debug: display(dfSmoothParams)

    # create sets for give index
    dfOriginalSet = dfsFF[rndindex].copy()
    dfAdjustedSet = dfsFFAdj[rndindex][['TRIPPURP','TLFTYPE','BIN','FF_ADJ']].copy()
    dfAdjustedSet.columns = (['TRIPPURP','TLFTYPE','BIN','FF'])
    dfSmoothedSet = dfAdjustedSet.copy() # copy of adjusted since this is where it starts
    
    # add column for round values
    dfOriginalSet['FF_CALIB_ROUND'] = rndindex
    dfAdjustedSet['FF_CALIB_ROUND'] = rndindex
    dfSmoothedSet['FF_CALIB_ROUND'] = rndindex

    dfOriginalSet['OAS'] = 'Original'
    dfAdjustedSet['OAS'] = 'Adjusted'
    dfSmoothedSet['OAS'] = 'Smoothed'

    if debug: display(dfAdjustedSet)

    # go through all trip purposes
    for tp in tps:
        if debug: print (tp)

        # make copy of filtered dataframe for working
        dfSmthSetTP = dfSmoothedSet[(dfSmoothedSet['TRIPPURP']==tp)].copy()

        # make copy of filtered manual adjustments for working
        dfSmthPrmForTP = dfSmoothParams[(dfSmoothParams['TRIPPURP']==tp)].copy()

        # smoothing is done row-by-row from the parameters file
        for index, row in dfSmthPrmForTP.iterrows():

            paramindex = index
            _binfrom = row['BIN_FROM']
            _binto   = row['BIN_TO']

            # show before if debug set to 1
            if(row['DEBUG_TOGGLE']==1):
                pd.set_option('display.max_rows', None)
                display(dfSmthSetTP[(dfSmthSetTP['BIN'].between(_binfrom,_binto))])
                pd.set_option('display.max_rows', 10)

            # linear adjustment
            if row['LINEAR_TOGGLE']:
                #print('linear')

                numPNTS = len(dfSmthSetTP[(dfSmthSetTP['BIN'].between(_binfrom,_binto))].index)
                bgnFFAdj = dfSmthSetTP[(dfSmthSetTP['BIN']==_binfrom)]['FF'].values[0]
                endFFAdj = dfSmthSetTP[(dfSmthSetTP['BIN']==_binto  )]['FF'].values[0]

                # interpolate with np
                interpolates = np.linspace(bgnFFAdj, endFFAdj, numPNTS)

                # display for debugging
                if(row['DEBUG_TOGGLE']==1):
                    display('Bgn Freq ' + str(bgnFFAdj))
                    display('End Freq ' + str(endFFAdj))
                    display('Size ' + str(numPNTS))
                    display('Interpolates ' + str(interpolates))

                dfSmthSetTP.loc[(dfSmthSetTP['BIN'].between(_binfrom,_binto)), 'FF'] = interpolates

            # moving average adjustment
            if row['MOVAVG_REACH']>0:
                
                maxbin = dfSmthSetTP.loc[dfSmthSetTP['BIN'].idxmax()]['BIN']
                
                # add zeros to keep moving average
                dfZeros = dfSmthSetTP.copy()
                dfZeros['BIN'] += maxbin + binsize
                dfZeros['FF'] = 0
                dfSmthSetTP = pd.concat([dfSmthSetTP, dfZeros], ignore_index=True)
                
                # reach is on either side, so average extents is reach 2 + 1 with min periods for the shoulders, shift moves the avergage to center on bin
                # moving average is calculated for entire dataset and then only applied to BIN_FROM-BIN_TO range
                dfRolling = dfSmthSetTP['FF'].rolling(row['MOVAVG_REACH']*2+1, min_periods=1).mean().shift(-1 * row['MOVAVG_REACH'])

                # debug
                if(row['DEBUG_TOGGLE']==1): display(dfRolling)
                    
                dfSmthSetTP.loc[dfSmthSetTP['BIN'].between(_binfrom,_binto),'FF'] = dfRolling                
                
                # trim back to maxbin
                dfSmthSetTP = dfSmthSetTP[dfSmthSetTP['BIN']<=maxbin]
            
            # adjust frequency by override
            if row['FF_OVERRIDE']>0:
                dfSmthSetTP.loc[(dfSmthSetTP['BIN'].between(_binfrom,_binto)),'FF'] = row['FF_OVERRIDE']

            # adjust frequency by relative amount
            dfSmthSetTP.loc[(dfSmthSetTP['BIN'].between(_binfrom,_binto)),'FF'] += row['FFADJ_REL']

            # set frequency to zero
            if row['FF_ZERO_TOGGLE']==1:
                dfSmthSetTP.loc[(dfSmthSetTP['BIN'].between(_binfrom,_binto)),'FF'] = 0

            # copy from previous round
            if row['NOADJ_TOGGLE']==1:
                dfNoAdj = dfOriginalSet[(dfOriginalSet['TRIPPURP']==tp)].copy()
                dfSmthSetTP.loc[(dfSmthSetTP['BIN'].between(_binfrom,_binto)),'FF'] = dfNoAdj[(dfNoAdj['BIN'].between(_binfrom,_binto))]['FF'].tolist()

            # show after if debug set to 1
            if(row['DEBUG_TOGGLE']==1):
                pd.set_option('display.max_rows', None)
                display(dfSmthSetTP[(dfSmthSetTP['BIN'].between(_binfrom,_binto))])
                pd.set_option('display.max_rows', 10)

            if debug: display(dfSmthSetTP)

        dfSmoothedSetHolder = pd.concat([dfSmoothedSetHolder, dfSmthSetTP], ignore_index=True)

    if debug: display(dfSmoothedSetHolder)

    # combine dataframes... smoothed with observed
    dfFFsForCharts = pd.concat([dfFFsForCharts,dfOriginalSet,dfAdjustedSet,dfSmoothedSetHolder], ignore_index=True)

if debug: display(dfFFsForCharts)

# MAKE INTERACTIVE CHART
py.offline.init_notebook_mode(connected=True)

lstCalibRnd  = dfFFsForCharts['FF_CALIB_ROUND'].unique().tolist() # iterative round names
lstTripPurp  = dfFFsForCharts['TRIPPURP'      ].unique().tolist() # HBW, HBShp, HBOth,...
lstOAS       = dfFFsForCharts['OAS'           ].unique().tolist() # Original, Adjusted, Smoothed

# multi select
selectCalibRnd  = widgets.SelectMultiple(options=lstCalibRnd, value=(lstCalibRnd[3],), description='Calib Round')
selectTripPurp  = widgets.SelectMultiple(options=lstTripPurp, value=('XI_HV',), description='Trip Purpose')
selectOAS       = widgets.SelectMultiple(options=lstOAS, value=('Original','Adjusted','Smoothed'), description='OAS')

# single select
selectPreset    = widgets.Select(options=dfZoomPresets['PRESET'].tolist(), value='Begin', description='Zoom Presets')

##custom extents
xmin = widgets.Text(value='0', description='X Min',style={'width':50, 'description_width': '30'})
xmax = widgets.Text(value='400', description='X Max',style={'width':50, 'description_width': '30'})
ymin = widgets.Text(value='0', description='Y Min',style={'width':'50', 'description_width': '30'})
ymax = widgets.Text(value='.2', description='Y Max',style={'width':50, 'description_width': '30'})

#container1 = widgets.HBox([selectCalibRnd, selectTripPurp, selectOAS, selectPreset])
#container2 = widgets.VBox([xmin, xmax, ymin, ymax])
#container3 = widgets.HBox([container1, container2])

#ui = container3

#out = widgets.interactive_output(update_plot_ff, {'calibrnds':selectCalibRnd, 'trippurps':selectTripPurp, 'oass':selectOAS, 'zoompreset':selectPreset, 'xmin':xmin, 'xmax':xmax, 'ymin':ymin, 'ymax':ymax})
#display(ui, out)

widgets.interactive(update_plot_ff, calibrnds=selectCalibRnd, trippurps=selectTripPurp, oass=selectOAS, zoompreset=selectPreset, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
#widgets.interactive(update_plot_ff, calibrnds=selectCalibRnd, trippurps=selectTripPurp, oass=selectOAS, zoompreset=selectPreset)


interactive(children=(SelectMultiple(description='Calib Round', index=(3,), options=(0, 1, 2, 3), value=(3,)),…

In [20]:
tp


'XI_HV'

In [21]:
dfSmoothParams

,TRIPPURP,BIN_FROM,BIN_TO,LINEAR_TOGGLE,MOVAVG_REACH,FF_OVERRIDE,FFADJ_REL,FF_ZERO_TOGGLE,NOADJ_TOGGLE,DEBUG_TOGGLE,NOTES
0,HBW,8,400,0,3,0.0000000,0.0000000,0,0,0,NaN
1,HBW,8,8,0,0,0.1760000,0.0000000,0,0,0,NaN
2,HBW,10,10,0,0,0.1465718,0.0000000,0,0,0,NaN
3,HBW,12,12,0,0,0.1256282,0.0000000,0,0,0,NaN
4,HBW,14,14,0,0,0.1150000,0.0000000,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11,HBSchSc,32,32,0,0,0.0000000,-0.0000030,0,0,0,NaN
12,HBSchSc,34,34,0,0,0.0000000,-0.0000030,0,0,0,NaN
13,HBSchSc,36,64,1,0,0.0000000,0.0000000,0,0,0,NaN
14,HBSchSc,62,62,0,0,0.0000000,-0.0000008,0,0,0,NaN


In [31]:
dfSmoothedSet[dfSmoothedSet['TRIPPURP']=='HBW']

,TRIPPURP,TLFTYPE,BIN,FF,FF_CALIB_ROUND,OAS
603,HBW,GC,0,0.8022745,3,Smoothed
604,HBW,GC,2,0.8021140,3,Smoothed
605,HBW,GC,4,1.0000000,3,Smoothed
606,HBW,GC,6,0.2922437,3,Smoothed
607,HBW,GC,8,0.1760004,3,Smoothed
...,...,...,...,...,...,...
799,HBW,GC,392,0.0000000,3,Smoothed
800,HBW,GC,394,0.0000000,3,Smoothed
801,HBW,GC,396,0.0000000,3,Smoothed
802,HBW,GC,398,0.0000000,3,Smoothed


In [22]:
# PREP FF FILE FOR NEXT MODEL RUN

# get last round data
lastRun = dfCalibrationRounds[dfCalibrationRounds.index==dfCalibrationRounds.shape[0]-1].index.values[0]
if debug: display(lastRun)

dfNextRoundFF = dfFFsForCharts[(dfFFsForCharts['FF_CALIB_ROUND']==lastRun) & (dfFFsForCharts['OAS']=='Smoothed')]
if debug: display(dfNextRoundFF)

# set initial bin to 1 -- DON'T USE THIS UNLESS YOU REALLY WANT TO
#dfNextRoundFF.loc[(dfNextRoundFF['BIN']==0),'FF'] = 1.0

# round to seven decimals 
dfNextRoundFF['BIN'] = round(dfNextRoundFF['BIN'],7)

dfNextRoundFF_Locked = pd.DataFrame()

# get locked friction factors
for index, row in dfCalibrationRounds.iterrows():
    trippurps = dfTripPurpLocks[dfTripPurpLocks['LOCK_FF_CALIB_ROUND_INDEX']==index]['TRIPPURP'].unique().tolist()
    dfLocked = dfFFsForCharts[(dfFFsForCharts['FF_CALIB_ROUND']==index) & (dfFFsForCharts['TRIPPURP'].isin(trippurps)) & (dfFFsForCharts['OAS']=='Smoothed')]
    dfNextRoundFF_Locked = pd.concat([dfNextRoundFF_Locked,dfLocked],ignore_index=True)
    if debug:
        print(index)
        print(trippurps)
        display (dfLocked)


#temp fix of duplicates
dfNextRoundFF_Locked = dfNextRoundFF_Locked.drop_duplicates()

dfNextRoundFF = pd.concat([dfNextRoundFF,dfNextRoundFF_Locked],ignore_index=True)

# set LT equal to MD
dfNextRoundFF_IXLT = dfNextRoundFF[dfNextRoundFF['TRIPPURP']=='IX_MD'].copy()
dfNextRoundFF_IXLT['TRIPPURP'] = 'IX_LT'
dfNextRoundFF_XILT = dfNextRoundFF[dfNextRoundFF['TRIPPURP']=='XI_MD'].copy()
dfNextRoundFF_XILT['TRIPPURP'] = 'XI_LT'
dfNextRoundFF_LT = dfNextRoundFF[dfNextRoundFF['TRIPPURP']=='MD'].copy()
dfNextRoundFF_LT['TRIPPURP'] = 'LT'

dfNextRoundFF = pd.concat([dfNextRoundFF,dfNextRoundFF_IXLT,dfNextRoundFF_XILT,dfNextRoundFF_LT],ignore_index=True)

dfNextRoundFF_Pivot = dfNextRoundFF.pivot(index=('BIN'), columns=('TRIPPURP'), values=('FF'))
dfNextRoundFF_Pivot.reset_index(inplace=True)
dfNextRoundFF_Pivot.rename(columns={'BIN':';BIN'},inplace=True)

colOrder = [';BIN','HBW','HBShp','HBOth','HBSchPr','HBSchSc','NHBW','NHBNW','LT','MD','HV','IX','IX_LT','IX_MD','IX_HV','XI','XI_LT','XI_MD','XI_HV']
dfNextRoundFF_Pivot = dfNextRoundFF_Pivot[colOrder]

path = os.path.join(dirIntermediate, str(lastRun) + '\\factors for next round\\FricFactor_AllPurp.csv')
dfNextRoundFF_Pivot.to_csv(path, index=False, float_format="%.7f")

display(path)
display(dfNextRoundFF_Pivot)

C:\Users\bhereth\AppData\Local\Temp\ipykernel_9844\4513930.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



'a:\\1 - TDM\\3 - Model Dev\\1 - WF\\2 - Sandbox\\v9.0Beta\\0 - CalibValid\\3 - Distribution\\_Friction_Factors\\3\\factors for next round\\FricFactor_AllPurp.csv'

TRIPPURP,;BIN,HBW,HBShp,HBOth,HBSchPr,HBSchSc,NHBW,NHBNW,LT,MD,HV,IX,IX_LT,IX_MD,IX_HV,XI,XI_LT,XI_MD,XI_HV
0,0,0.8022745,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,0.60,1.0000000,1.0000000,1.00000000,0.8000000,1.0000000,1.0000000,1.0000000
1,2,0.8021140,0.9973000,0.8516710,1.0000000,0.9192479,0.9563057,0.9980000,0.3331734,0.3331734,0.3065848,0.59,0.9300000,0.9300000,0.98466667,0.7850000,0.9800000,0.9800000,0.9000000
2,4,1.0000000,0.7380811,0.6224852,1.0000000,0.6719900,0.8400000,0.6722939,0.1163827,0.1163827,0.1423449,0.55,0.4618117,0.4618117,0.96933333,0.7000000,0.9500000,0.9500000,0.6750000
3,6,0.2922437,0.4301406,0.1954229,0.5245212,0.0833608,0.5350000,0.3445940,0.0347007,0.0347007,0.0886943,0.45,0.1674645,0.1674645,0.95400000,0.5028894,0.9017035,0.9017035,0.3500000
4,8,0.1760000,0.2228886,0.0806292,0.1215014,0.0223937,0.3510495,0.1797357,0.0090372,0.0090372,0.0657937,0.09,0.0650000,0.0650000,0.93866667,0.2000000,0.8534069,0.8534069,0.3266178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,392,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.00,0.0000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000,0.0000145
197,394,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.00,0.0000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000,0.0000000
198,396,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.00,0.0000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000,0.0000000
199,398,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.00,0.0000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000,0.0000000


In [23]:
import matplotlib.pyplot as plt
import numpy as np

m = widgets.FloatSlider(min=-5,max=5,step=0.5, description="Slope")
c = widgets.FloatSlider(min=-5,max=5,step=0.5, description="Intercept")

# An HBox lays out its children horizontally
ui = widgets.HBox([m, c])

def plot(m, c):
    x = np.random.rand(10)
    y = m *x + c
    plt.plot(x,y)
    plt.show()

out = widgets.interactive_output(plot, {'m': m, 'c': c})

display(out, ui)

Output()